In [1]:
import numpy as np
import time
import hashlib
import glob
import os
import progressbar
import cv2

In [2]:
from auto_pose.ae.pysixd_stuff import transform
from auto_pose.ae.pysixd_stuff import view_sampler
from auto_pose.ae.utils import lazy_property

In [ ]:
class Dataset(object):
    
    def __init__(self, dataset_path, **kw):
        self.shape = (int(kw['h']), int(kw['w']), int(kw['c']))
        self.noof_training_imgs = int(kw['noof_training_imgs'])
        self.dataset_path = dataset_path
        
        self.bg_img_paths = glob.glob(kw['background_images_glob'])
        self.noof_bg_imgs = min(int(kw['noof_bg_imgs']), len(self.bg_img_paths))
        
        self._kw = kw
        # self._aug = eval(self._kw['code'])

        self.train_x = np.empty( (self.noof_training_imgs,) + self.shape, dtype=np.uint8 )
        self.mask_x = np.empty( (self.noof_training_imgs,) + self.shape[:2], dtype= bool)
        self.noof_obj_pixels = np.empty( (self.noof_training_imgs,), dtype= bool)
        self.train_y = np.empty( (self.noof_training_imgs,) + self.shape, dtype=np.uint8 )
        self.bg_imgs = np.empty( (self.noof_bg_imgs,) + self.shape, dtype=np.uint8 )
        if np.float(eval(self._kw['realistic_occlusion'])):
            self.random_syn_masks